In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import os

#variable
SUMMARY = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/LR_SummaryTest_Table.tsv'
TSS = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Merged_Causality_TSS_TSS.tab'
ANNOT =  "/storage/mgymrek/gtex/annotations/encode/"
ENHANC = ["Strong_Enhancer","Weak_Enhancer"]

sep='"\t"'
command = "cat %s |awk -F'\t' '{print $1%s$2%s$75%s$4}' > all_analyzed_strs.tab"%(SUMMARY, sep,sep,sep)
value = os.system(command)

In [19]:
#All STRs
allstrs = pd.read_csv(SUMMARY, sep='\t')
allstrs['locus']=allstrs.apply(lambda x: x["chrom"]+":"+str(x["str.start"]), 1)
estrs = allstrs.loc[allstrs['E.tissues']>=1] 
print(allstrs.shape, 'All loci:', len(set(allstrs['locus'])) , estrs.shape, 'eSTR loci:',len(set(estrs['locus'])))


#enhancers
enhancer = {
    'Strong_E': os.path.join(ANNOT,"%s_GM12878.bed"%ENHANC[0]),
    'Weak_E': os.path.join(ANNOT,"%s_GM12878.bed"%ENHANC[1])
            }
#intersect with strong enhancers
command = "bedtools intersect -a all_analyzed_strs.tab -b %s > Intersect.bed"%enhancer['Strong_E']
value = os.system(command)
enhancer['inter_strong'] = pd.read_csv('Intersect.bed', sep='\t', names=["chrom","start","end","gene"]).drop_duplicates()
feature_loci = set(enhancer['inter_strong'].apply(lambda x: x["chrom"]+":"+str(x["start"]), 1))
allstrs['inter_strong'] = np.where(allstrs["locus"].isin(feature_loci), 1, 0)


#intersect with weak enhancers
command = "bedtools intersect -a all_analyzed_strs.tab -b %s > Intersect.bed"%enhancer['Weak_E']
value = os.system(command)
enhancer['inter_weak'] = pd.read_csv('Intersect.bed', sep='\t', names=["chrom","start","end","gene"]).drop_duplicates()
feature_loci = set(enhancer['inter_weak'].apply(lambda x: x["chrom"]+":"+str(x["start"]), 1))
allstrs['inter_weak'] = np.where(allstrs["locus"].isin(feature_loci), 1, 0)

(307187, 78) All loci: 125758 (25619, 78) eSTR loci: 20192


In [32]:
enhan = allstrs.loc[(allstrs['inter_strong']==1)|(allstrs['inter_weak']==1)].copy()
print('All STRs at enhancers', len(set(enhan['locus'])), enhan.loc[enhan['E.tissues']>=1].shape,'eSTRs at enhancers',len(set(enhan.loc[enhan['E.tissues']>=1]['locus'])) )

Eenhan = enhan.loc[enhan['E.tissues']>=1].copy()
Motifs = list(set(Eenhan['motif']))

locus=len(set(allstrs['locus']))
eSTR=len(set(estrs['locus']))
locus_enh = len(set(enhan['locus']))
estr_enh_locus = len(set(enhan.loc[enhan['E.tissues']>=1]['locus']))  
odd_enhancer, pv_enhancer = ss.fisher_exact([[estr_enh_locus,eSTR-estr_enh_locus],[locus_enh-estr_enh_locus, locus-eSTR-(locus_enh-estr_enh_locus)]])
print('Enrichment of eSTRs at enhancer regions',odd_enhancer, pv_enhancer)

odds=[]
pval=[]
count=[]

#overall enrichment #66 eSTRs motifs 
for motif in Motifs:
    me = Eenhan.loc[Eenhan['motif']==motif].shape[0]
    mne= Eenhan.shape[0] - me
    nme= enhan.loc[enhan['motif']==motif].shape[0]-me
    nmne=enhan.shape[0]-enhan.loc[enhan['motif']==motif].shape[0]-(mne) 
    #print([me,mne],[nme,nmne])
    
    OR, p = ss.fisher_exact([[me,mne],[nme,nmne]])
    
    odds.append(np.log2(OR))
    pval.append(-np.log10(p))
    count.append((enhan.loc[enhan['motif']==motif].shape[0], me))

data=pd.DataFrame.from_dict({'motif':Motifs, 'OR':odds, 'pvalue':pval, 'Counts':count}).sort_values('pvalue')
data.index=list(data['motif'])

All STRs at enhancers 7924 (2204, 80) eSTRs at enhancers 1673
Enrichment of eSTRs at enhancer regions 1.4353035443698845 2.105432869238656e-34


In [33]:
plt.figure()
plt.bar(data['pvalue'])

TypeError: bar() missing 1 required positional argument: 'height'